In [1]:
import numpy as np
import os, sys
from datetime import datetime
import requests
from requests.exceptions import HTTPError
%load_ext autoreload
%autoreload 2

In [2]:
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.model.center_of_mass import CenterOfMass
from utilities.model.structure import Structure
from sqlalchemy import func
from sql_setup import session

In [3]:
def get_transformation_matrix(animal):
    try:
        url = f'https://activebrainatlas.ucsd.edu/activebrainatlas/rotation/{animal}/manual/2'
        response = requests.get(url)
        response.raise_for_status()
        # access JSOn content
        transformation_matrix = response.json()

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

    return transformation_matrix

In [4]:
prep_id = 'DK52'

In [5]:
transformation = get_transformation_matrix(prep_id)
r = np.array(transformation['rotation'])
t = np.array(transformation['translation'])

In [6]:
r

array([[ 0.90823119, -0.24441544, -0.05697743],
       [ 0.2488724 ,  0.90471918,  0.08611017],
       [ 0.03237074, -0.09804858,  0.93659374]])

In [7]:
t

array([[-15509.17500957],
       [-12625.10609468],
       [   -66.471405  ]])

In [8]:
# person_id = 2 = beth
rows = session.query(CenterOfMass).filter(
    CenterOfMass.active.is_(True))\
        .filter(CenterOfMass.prep_id == prep_id)\
        .filter(CenterOfMass.person_id == 2)\
        .all()
row_dict = {}
for row in rows:
    structure = row.structure.abbreviation
    row_dict[structure] = [row.x, row.y, row.section]

In [9]:
def brain_to_atlas_transform(
    brain_coord, r, t,
    brain_scale=(0.325, 0.325, 20),
    atlas_scale=(10, 10, 20)
):
    """
    Takes an x,y,z brain coordinates, and a rotation matrix and transform vector.
    Returns the point in atlas coordinates.
    
    The recorded r, t is the transformation from atlas to brain such that:
        t_phys = brain_scale @ t
        brain_coord_phys = r @ atlas_coord_phys + t_phys

    And so we have the following reverse transformation:
        r_inv = inv(r)
        atlas_coord_phys = r_inv @ brain_coord_phys - r_inv @ t_phys
    """
    brain_scale = np.diag(brain_scale)
    atlas_scale = np.diag(atlas_scale)

    # Transform brain coordinates to physical space
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    brain_coord_phys = brain_scale @ brain_coord
    
    # Apply affine transformation in physical space
    t_phys = brain_scale @ t
    r_inv = np.linalg.inv(r)
    atlas_coord_phys = r_inv @ brain_coord_phys - r_inv @ t_phys
    
    # Bring atlas coordinates back to atlas space
    atlas_coord = np.linalg.inv(atlas_scale) @ atlas_coord_phys

    return atlas_coord.T[0] # Convert back to a row vector

In [10]:
data = []
person_id = 28 # bili, this is your database ID
for abbrev,v in row_dict.items():
    x = v[0]
    y = v[1]
    section = v[2]
    structure = session.query(Structure).filter(Structure.abbreviation == func.binary(abbrev)).one()
    x,y,section = brain_to_atlas_transform([[x],[y],[section]], r, t)
    data.append([x, y, section])
    print(abbrev, x, y, section)
    continue
    com = CenterOfMass(
        prep_id=prep_id, structure=structure, x=x, y=y, section=section,
        created=datetime.utcnow(), active=True, person_id=person_id, input_type='aligned'
    )
    try:
        session.add(com)
        session.commit()
    except Exception as e:
        print(f'No merge for {abbrev} {e}')
        session.rollback()

data = np.array(data)

10N_L 2327.168186349307 397.0555881691571 284.29689283677834
10N_R 2334.3633121835855 386.7935632075005 314.59867097731797
12N 2337.281349934282 419.0353390341218 302.3557967619031
3N_L 1963.028625873284 439.94689937538305 307.7824762460805
3N_R 1956.2907782280572 437.6258047142414 319.52210687011825
4N_L 2002.8028606776597 421.9844237995146 296.5456315403014
4N_R 2004.7897444023765 417.5447562723411 321.90368058676336
5N_L 2114.8442588432745 527.2254682969826 245.6654449893141
5N_R 2118.546004235183 454.7681133596979 388.0835605113692
6N_L 2181.043995003098 477.22400972095767 332.65860234837214
6N_R 2178.1957551667956 496.1424433562685 294.19321884820977
7N_L 2320.4878427877493 489.5687537991928 384.27996139797483
7N_R 2270.5617146141744 546.4577428259047 390.25588781775355
AP 2342.3894006463825 362.445513549831 295.03463771742815
Amb_L 2313.562570345002 557.9028006546095 243.83712251262887
DC_L 2200.39017770621 440.5235329591533 194.80551358126246
DC_R 2211.1162450898905 344.72831536

In [11]:
# Print atlas COMs for comparison

from utilities.sqlcontroller import SqlController

def get_atlas_centers(
        atlas_box_size=(1000, 1000, 300),
        atlas_box_scales=(10, 10, 20),
        atlas_raw_scale=10
):
    atlas_box_scales = np.array(atlas_box_scales)
    atlas_box_size = np.array(atlas_box_size)
    atlas_box_center = atlas_box_size / 2
    sqlController = SqlController('Atlas')
    atlas_centers = sqlController.get_centers_dict('Atlas')

    for structure, center in atlas_centers.items():
        # transform into the atlas box coordinates that neuroglancer assumes
        center = atlas_box_center + np.array(center) * atlas_raw_scale / atlas_box_scales
        atlas_centers[structure] = center

    return atlas_centers

get_atlas_centers()

No histology for Atlas
No scan run for Atlas


{'10N_L': array([703.341 , 547.9286, 135.6302]),
 '10N_R': array([703.341 , 547.9286, 164.3698]),
 '12N': array([697.227 , 567.0988, 149.25  ]),
 '3N_L': array([364.555 , 391.506 , 143.2358]),
 '3N_R': array([364.555 , 391.506 , 156.7642]),
 '4N_L': array([399.441  , 396.104  , 137.62875]),
 '4N_R': array([399.441  , 396.104  , 162.37125]),
 '5N_L': array([456.7992 , 502.07948,  81.9005 ]),
 '5N_R': array([456.7992 , 502.07948, 218.0995 ]),
 '6N_L': array([517.1597, 512.4922, 132.3135]),
 '6N_R': array([517.1597, 512.4922, 167.6865]),
 'AP': array([702.306 , 525.6525, 151.    ]),
 'Amb_L': array([612.44  , 603.826 ,  88.2695]),
 'Amb_R': array([612.44  , 603.826 , 211.7305]),
 'DC_L': array([583.2885, 456.4954,  42.084 ]),
 'DC_R': array([583.2885, 456.4954, 257.916 ]),
 'IC': array([496.7529, 251.693 , 150.    ]),
 'LC_L': array([496.19734, 442.8079 , 108.22515]),
 'LC_R': array([496.19734, 442.8079 , 191.77485]),
 'LRt_L': array([688.194, 646.863,  93.755]),
 'LRt_R': array([688.194,